In [1]:
import pandas as pd

In [2]:
path='/content/info.csv'

In [8]:
df=pd.read_csv(path)

In [9]:
df.head(2)

,Question,Answer
0,How can I apply for admission at Superior Univ...,"To apply on-campus, visit the admission office..."
1,How can I apply online for admission at Superi...,"To apply online, go to Superior University's o..."


In [4]:
!pip install  sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [10]:
embeddings = model.encode(df['Answer'].values)

In [11]:
import numpy as np
embeddings=np.array(embeddings)

In [12]:
np.save('QaNbot.npy',embeddings)

In [13]:
embeddings=np.load('QaNbot.npy')

In [14]:
#FAISS
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 64.6 MB/s eta 0:00:00


In [15]:
import faiss
dimensions=embeddings.shape[1]
faiss_index=faiss.IndexFlatL2(dimensions)
faiss_index.add(embeddings)

In [16]:
faiss.write_index(faiss_index,'faiss_index.faiss')

In [23]:
def get_info(query,model,faiss_index,count=1):
  query_embedding=model.encode([query])
  distance,indices=faiss_index.search(query_embedding,count)
  for i in range(count):
    print(f'Question{i+1}')
    print(f"Distance:{distance[0][i]}")
    print(df['Answer'].iloc[indices[0][i]])


In [22]:
get_info("How can I apply for admission at Superior University on-campus?",model,faiss_index,1)

Question1
Distance:0.2754945755004883
To apply on-campus, visit the admission office at your chosen Superior University campus. There you can consult with admissions staff, pay the prospectus/application fee, and complete the application form. You will then receive an admission test/interview slip and must take the university's entry test (and interview, if required). Once you pass, you will get an offer letter and a fee challan. Finally, submit the paid fee slip along with your academic documents to complete the enrollment process.


In [25]:
get_info("when admissions open",model,faiss_index,1)

Question1
Distance:0.5227386951446533
Superior University typically has two intakes: Fall and Spring. The exact deadlines vary by year and program. For example, Spring 2025 admissions are currently open:contentReference[oaicite:2]{index=2}, with classes likely starting in January/February 2025. Fall admissions usually open in mid-year for classes starting around August/September (for instance, Fall 2024 classes began in October 2024). Admissions usually close a few weeks before classes start. Check the official website or contact admissions each semester for the final application closing dates, as they can change.
